In [51]:
import tweepy
import pandas as pd
import os
import unidecode

In [15]:
sel = pd.read_csv('../Data/SEL/SEL_full.txt', delimiter='\t', encoding= 'ISO-8859-1')

In [62]:
emociones = list(sel['Categoría'].unique())
emociones_dict ={}
for emocion in emociones:
    emociones_dict[emocion] = list(sel[sel['Categoría'] == emocion].sort_values(' PFA',  ascending=False).iloc[0:10]['Palabra'])
    for palabra in emociones_dict[emocion]:
        if not palabra.isascii():
            emociones_dict[emocion].append(unidecode.unidecode(palabra))

In [122]:
client = tweepy.Client(bearer_token = "AAAAAAAAAAAAAAAAAAAAAKKEcwEAAAAA1VgzT5upnu%2B5POQ3dbdUx0%2B5vfA%3DdslAW6ut31gZhQD9SjNKJ4scCFd8bHgbevp8eGcJprbd2LTrGf", wait_on_rate_limit = True)

In [124]:
def get_tweets(emotion, words_list, number_tweets, start, end):
    emotion_words = '(' +' OR '.join(words_list) + ')'
    searched_tweets = [tweet for tweet in tweepy.Paginator(client.search_all_tweets, query = emotion_words + ' place_country:CO' ,start_time = start, end_time = end, 
                                max_results=500).flatten(limit=number_tweets)]
    tweets_df = pd.DataFrame(searched_tweets, columns = ['text'])
    tweets_df['emotion']= emotion
    return tweets_df

In [ ]:
start =  "2021-03-29T11:59:59Z"
end = "2021-03-30T11:59:59Z"
df_list = []
for i in emociones_dict.keys():
    print(i)
    df_list.append(get_tweets(i, emociones_dict[i], 10, start, end))
emociones_df = pd.concat(df_list)